In [ ]:
!pip install -q pyswarms

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 2.1 MB/s eta 0:00:00


In [ ]:
import pyswarms as ps

In [ ]:
!pip install -q geneticalgorithm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from geneticalgorithm import geneticalgorithm as ga

In [ ]:
import numpy as np
import pandas as pd

from numpy import mean
from numpy import std

from IPython.core.pylabtools import figsize

from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, KFold
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix as CM
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, KFold
from sklearn.svm import SVR
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import neighbors

2023-11-03 00:35:53,279 - numexpr.utils - INFO - NumExpr defaulting to 2 threads.


In [ ]:
def GS_Anneal(model,h_names,h1,h2, xtrain, ytrain,xtest, ytest): # h_names means the names of the hyperparameters
  # Here we have an implementation of Simmulated Annealing in Two Dimensions
  MIN_IND1 = 0
  MAX_IND1 = len(h1)
  MIN_IND2 = 0
  MAX_IND2 = len(h2)
  STEP_SIZE = 7    # Std of gaussian steps
  COOLING = 0.98    # T_{n+1} = T_{n}*COOLING, i.e. geometric cooling
  def cost_function(model):
    #pipe = Pipeline([['Scale',scale],['Model',model]])
    #k_fold = cross_val_score(estimator=model,X=x,y=y,cv=10,scoring='neg_mean_squared_error')
    model.fit(xtrain,ytrain)
    return 1 - model.score(xtest,ytest)


  def random_start():
      i = np.random.randint(MIN_IND1,MAX_IND1-1)
      j = np.random.randint(MIN_IND2,MAX_IND2-1)
      pos = [i,j]
      return pos


  def random_neighbour(pos,T):
      i = pos[0]
      j = pos[1]
      raw_i = np.random.normal()*STEP_SIZE*T
      delta_i = np.floor(raw_i).astype(int)
      raw_j = np.random.normal()*STEP_SIZE*T
      delta_j = np.floor(raw_j).astype(int)
      if (delta_i == 0) & (delta_j == 0):
        delta_i = np.sign(raw_i).astype(int)
        delta_j = np.sign(raw_j).astype(int)
      new_i = min(MAX_IND1-1, max(MIN_IND1, i+delta_i))
      new_j = min(MAX_IND2-1, max(MIN_IND2, j+delta_j))
      new_pos = [new_i,new_j]
      return new_pos


  def acceptance_probability(cost, new_cost, temperature):
      if new_cost < cost:
          return 1
      else:
          p = np.exp(-(new_cost - cost) / temperature)
          return p


  def locate_min(a):
      smallest = min(a)
      return smallest, [index for index, element in enumerate(a) if smallest == element]

  def annealing(pos0 = random_start()):
      # Start with a random configuration
      # alpha = random_start()
      i = pos0[0]
      j = pos0[1]
      # All layers will have the same number of nodes (keep the example from getting too complicated)
      model.set_params(**{list(h_names)[0]:h1[i-1],list(h_names)[1]:h2[j-1]})

      # Compute the cost function (in this case, 1-accuracy)
      cost = cost_function(model)

      # Keep lists of results, start with the current
      states1, states2, costs = [h1[i-1]], [h2[j-1]], [cost]
      T = 1
      pos = pos0
      while T > 1e-10:
          new_pos = random_neighbour(pos,T)
          i = new_pos[0]
          j = new_pos[1]
          model.set_params(**{list(h_names)[0]:h1[i-1],list(h_names)[1]:h2[j-1]})
          new_cost = cost_function(model)
          if acceptance_probability(cost, new_cost, T) > np.random.random():
              cost = new_cost
              pos = new_pos
              states1.append(h1[pos[0]])
              states2.append(h2[pos[1]])
              costs.append(cost)
          T = T*COOLING
      return model,np.column_stack([states1,states2]),costs
  m,states,costs = annealing()
  return states, costs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv('drive/MyDrive/mobile.csv')

Mounted at /content/drive


In [ ]:
X = data.loc[:,'battery_power':'wifi'].values
y = data['price_range'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)

In [ ]:
X.shape

(2000, 20)

In [ ]:
scale = StandardScaler()
if len(X_train.shape)==1:
  X_train = X_train.reshape(-1,1)
  X_test = X_test.reshape(-1,1)
else:
  X_train = scale.fit_transform(X_train)
  X_test = scale.transform(X_test)

PART 1:

Log Reg:

In [ ]:
model = LogisticRegression(solver='lbfgs', max_iter=10000)
model.fit(X_train,y_train)
predicted_classes = model.predict(X_test)
accuracy = accuracy_score(y_test,predicted_classes)
logistic=CM(y_test, predicted_classes)
print(logistic)
print(accuracy)

[[117   2   0   0]
 [  8 116   4   0]
 [  0   4 129   4]
 [  0   0   3 113]]
0.95


Poly Kernal

In [ ]:
best_poly_acc=0
best_poly_degree=0
best_poly_c=0

for degree in range(1,4):
  for c in range(1,151):
    model = svm.SVC(kernel='poly',C=c/2,degree=degree)
    model.fit(X_train,y_train)
    predicted_classes=model.predict(X_test)
    accuracy=accuracy_score(y_test, predicted_classes)
    if accuracy> best_poly_acc:
      best_poly_acc=accuracy
      best_poly_degree=degree
      best_poly_c=c

model = svm.SVC(kernel='poly',C=best_poly_c,degree= best_poly_degree)
model.fit(X_train,y_train)
predicted_classes= model.predict(X_test)
accuracy= accuracy_score(y_test, predicted_classes)
logistic=CM(y_test, predicted_classes)
print(logistic)
print(accuracy)

[[118   1   0   0]
 [  2 121   5   0]
 [  0   3 133   1]
 [  0   0   5 111]]
0.966


RBF

In [ ]:
h1= np.arange(0.0005,0.0101,0.0001)
h2=np.arange(5,75.05,0.5)
h_names = ['gamma','C']
model = svm.SVC(kernel='rbf',probability=True)

In [ ]:
states, costs= GS_Anneal(model,h_names, h1, h2, X_train, y_train, X_test, y_test)

In [ ]:
costs=costs
states=states.tolist()

best_rbf_c=0
best_rbf_degree=0
best_rbf_acc=0
for i in range (len(costs)):
  if costs[i]>best_rbf_acc:
    best_rbf_acc= costs[i]
    best_rbf_c= states[i][1]
    best_rbf_degree=states[i][0]

model = svm.SVC(kernel='rbf', C=best_rbf_c,gamma=best_rbf_degree,probability=True)
fit= model.fit(X_train,y_train)
predicted_classes= model.predict(X_test)
accuracy= accuracy_score(y_test, predicted_classes)
logistic=CM(y_test, predicted_classes)
print(logistic)
print(accuracy)

[[114   5   0   0]
 [ 10 113   5   0]
 [  0   5 127   5]
 [  0   0   5 111]]
0.93


K nearest neighbor

In [ ]:
best_knn_acc=0
best_knn_degree=0
best_knn_c=''

for weights in ['uniform', 'distance']:
  for degree in range(3,126):
    model = neighbors.KNeighborsClassifier(degree, weights=weights)
    model.fit(X_train,y_train)
    predicted_classes= model.predict(X_test)
    accuracy= accuracy_score(y_test, predicted_classes)
    if accuracy> best_knn_acc:
      best_knn_degree= degree
      best_knn_acc=accuracy
      best_knn_c=weights

model = neighbors.KNeighborsClassifier(best_knn_degree, weights=best_knn_c)
model.fit(X_train,y_train)
predicted_classes= model.predict(X_test)
accuracy= accuracy_score(y_test, predicted_classes)
logistic=CM(y_test, predicted_classes)
print(logistic)
print(accuracy)

[[96 23  0  0]
 [33 75 20  0]
 [ 0 25 93 19]
 [ 0  1 22 93]]
0.714


PART 2:

Log Reg:

In [ ]:
cv = KFold(n_splits=5, random_state=123, shuffle=True)
model=LogisticRegression(solver="lbfgs", max_iter=10000)
model.fit(X_train,y_train)
scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy:'+str(mean(scores)))

Accuracy:0.9506666666666665


Poly Kernal

In [ ]:
cv = KFold(n_splits=5, random_state=123, shuffle=True)
model=svm.SVC(kernel='poly',C=best_poly_c,degree= best_poly_degree)
model.fit(X_train,y_train)
scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
print("Hyperparameters: C = "+ str(best_poly_c)+", degree = "+str(best_poly_degree))
print('Accuracy: '+str(mean(scores)))

Hyperparameters: C = 143, degree = 1
Accuracy: 0.9573333333333334


RBF

In [ ]:
cv = KFold(n_splits=5, random_state=123, shuffle=True)
model=svm.SVC(kernel='rbf', C=best_rbf_c,gamma=best_rbf_degree,probability=True)
model.fit(X_train,y_train)
scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
print("Hyperparameters: C = "+ str(best_rbf_c)+", gamma = "+str(best_rbf_degree))
print('Accuracy: mean is '+str(mean(scores)))

Hyperparameters: C = 61.0, gamma = 0.0005
Accuracy: mean is 0.9253333333333333


K nearest neighbor

In [ ]:
cv = KFold(n_splits=5, random_state=123, shuffle=True)
model = neighbors.KNeighborsClassifier(best_knn_degree, weights=best_knn_c)
fit= model.fit(X_train,y_train)
scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
print("Hyperparameters: weights = "+ str(best_knn_c)+", number of neighbors = "+str(best_knn_degree))
print('Accuracy: mean is '+str(mean(scores)))

Hyperparameters: weights = distance, number of neighbors = 119
Accuracy: mean is 0.6366666666666666
